In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

## dataset

In [ ]:
# TODO upload your dataset or use roboflow download code
download_url="https://app.roboflow.com/ds/RaE05sWFT4?key=xUGZbmfMyK"
!curl -L {download_url} > roboflow.zip
!unzip ./roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   903  100   903    0     0   2791      0 --:--:-- --:--:-- --:--:--  2787
100 12.0M  100 12.0M    0     0  9825k      0  0:00:01  0:00:01 --:--:-- 9825k
Archive:  ./roboflow.zip
  inflating: README.dataset.txt      
  inflating: README.roboflow.txt     
  inflating: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/fall_mp4-0003_jpg.rf.7754d8fbf40e33514f9f8bef765e32a0.jpg  
 extracting: test/images/fall_mp4-0012_jpg.rf.d7d696544297662b96ebc08cf5e10dd9.jpg  
 extracting: test/images/fall_mp4-0027_jpg.rf.788a95bcd8c44412808d092a2b9c2613.jpg  
 extracting: test/images/fall_mp4-0042_jpg.rf.583bee9eaacb474fd40676b0e7b65867.jpg  
 extracting: test/images/fall_mp4-0076_jpg.rf.7856f7085901cba487f2ba325b671c18.jpg  
 extracting: test/images/fall_mp4-0082_jpg.rf.503a4c61a575d65c01df53f9b6ece

### train model

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 45.5 MB/s eta 0:00:00


In [ ]:
yaml_path = 'data.yaml'

In [ ]:
# TODO train the model with your dataset
# recommend at least train 50 epochs
!yolo detect train data={yaml_path} model=yolov8n.pt epochs=50 imgsz=640

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 6.25M/6.25M [00:00<00:00, 115MB/s]
Ultralytics 8.3.50 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=

#### test model

In [ ]:
from ultralytics import YOLO
# TODO test your model with test img in dataset
best_model_path="/content/runs/detect/train/weights/best.pt"
test_img="/content/valid/images/fall_mp4-0000_jpg.rf.d5f33f0f6eeac6b8709821908b558500.jpg"
model = YOLO(best_model_path)
model.predict(test_img, save=True, imgsz=640, conf=0.5)

FileNotFoundError: /content/valid/images/fall_mp4-0000_jpg.rf.d5f33f0f6eeac6b8709821908b558500.jpg does not exist

## export yolov8 to trt

In [ ]:
from pathlib import Path
# TODO your train dir ex. runs/detect/train/
train_path = Path("/content/runs/detect/train")
model_path = train_path / "weights"
pt_file = str(model_path / "best.pt")
onnx_file = str(model_path / "best.onnx")
trt_file = str(model_path / "best.engine")

In [ ]:
# tensorRT engine
!yolo export model={pt_file} format=onnx

Ultralytics 8.3.50 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 277.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 285.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 267.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 206.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 285.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.6s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime']
requirements: ⚠️ Restart runtime or rerun command for updates

In [ ]:
# download onnx
from google.colab import files
files.download(onnx_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>